<a href="https://colab.research.google.com/github/isowiki/mi-primer-repositorio-web-/blob/main/notebooks/07crmbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 Segmentación de Clientes con Modelo RFM

En este ejercicio vamos a transformar datos transaccionales crudos en segmentos de clientes accionables.

**Pasos:**
1. Generación, Carga y Exploración de Datos
2. Cálculo de Métricas RFM
3. Scoring
4. Segmentación Estratégica para la Toma de Decisiones
5. Visualización

In [7]:
import pandas as pd
import numpy as np
import datetime


In [8]:
np.random.seed(42)
n_transacciones = 1000
n_clientes = 50


In [9]:
fechas = pd.date_range(start='2023-01-01', end='2023-12-31', periods=n_transacciones)
fechas = np. random.choice(fechas, n_transacciones) # Desordenar fechas
cliente_ids = np. random. randint (1000, 1000 + n_clientes, n_transacciones)
cantidades = np. random.randint(1, 10, n_transacciones)
precios = np.round(np. random.uniform(10, 100, n_transacciones), 2)


In [10]:

 df = pd. DataFrame({
'ID_Transaccion': [f'TRX-{i}' for i in range(n_transacciones)],
'ID_Cliente': cliente_ids,
'Fecha': fechas,
'Cantidad': cantidades,
'Precio_Unitario': precios

 })
df[ 'Total'] = df[ 'Cantidad'] * df[ 'Precio_Unitario']